# Implementing CNN on Cifar10 dataset using PyTorch

# Load libraries

In [4]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [5]:
# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download datasets

In [7]:
train_data = datasets.CIFAR10(root='./cifar10', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10(root='./cifar10', train=False, transform=transform, download=True)

100.0%


In [8]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Build the architecture

In [9]:
class CnnCifar10(nn.Module):
    def __init__(self):
        super().__init__()
        #feature extractor
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.cnn2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)
        self.pool2 = nn.MaxPool2d(2, stride=2)

        #classifier
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=6 * 6 * 16, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=2048)
        self.fc3 = nn.Linear(in_features=2048, out_features=1024)
        self.fc4 = nn.Linear(in_features=1024, out_features=512)
        self.fc5 = nn.Linear(in_features=512, out_features=256)
        self.fc6 = nn.Linear(in_features=256, out_features=10)

    def forward(self, x):
        x = self.cnn1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)

        x = self.pool2(nn.ReLU()(self.cnn2(x)))

        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        x = nn.ReLU()(x)
        x = self.fc4(x)
        x = nn.ReLU()(x)
        x = nn.ReLU()(x)
        x = self.fc5(x)
        x = nn.ReLU()(x)
        x = self.fc6(x)
        return x

In [10]:
model = CnnCifar10()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Train

In [11]:
for epoch in range(5):
    for images, labels in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

TypeError: conv2d() received an invalid combination of arguments - got (int, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
